# SPDE

In this tutorial, we show how to use the API SPDE.

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
AA=gl.AFunction()
AA.eval(5)

In [ ]:

class Myfunc(gl.AFunction):
    def __init__(self):
        pass
    def eval(self,x):
        return np.log(x)
    
a=Myfunc()
plt.plot(np.array([a.eval(i) for i in range(1,130)]))

ch = gl.Chebychev()
#ch.fit2(a,1,130)
Myfunc.__bases__

In [ ]:
dat = gl.Db.create()
ndat = 2
np.random.seed(123)
dat["x"] = np.random.uniform(size=ndat)
dat["y"] = np.random.uniform(size=ndat)
dat["Z"] = np.random.normal(size=ndat)
dat.setLocators(["x","y"],gl.ELoc.X)
dat.setLocators(["Z"],gl.ELoc.Z)

grid = gl.DbGrid.create([150,150],[0.01,0.01],[-0.25,-0.25])
model = gl.Model.createFromParam(gl.ECov.BESSEL_K,param=1,range=0.5,sill=0.5)
model.addCovFromParam(gl.ECov.NUGGET,sill=0.3)

spde = gl.SPDE()
spde.init(model,grid,dat,gl.ESPDECalcMode.KRIGING)
        

In [ ]:
spde.compute()

In [ ]:
spde.query(grid)

In [ ]:
gp.plot(grid)

In [ ]:
spde.computeLogDet(10,0)

In [ ]:
Q = spde.getPrecisionOp(0)
print(Q.computeLogDet(2,0))
Q.getPolyCoeffs(gl.EPowerPT.ONE)

In [ ]:
#S = Q.getShiftOp()
#u = np.linspace(0,S.getMaxEigenValue(),100)
#(2*(1 + 2 * u + u**2)).max()

In [ ]:
pmc = spde.getPrecisionKriging()
cheb = gl.Chebychev()
pmc.preparePoly(cheb)
cheb.display()

In [ ]:
u = np.linspace(1,23635,200)
y = np.array([cheb.eval(i) for i in u])
y0 = np.array([np.log(i) for i in u])
plt.plot(u,y)
plt.plot(u,y0,"--")
plt.plot((y**2-y0**2)/(y0**2+1e-2))
np.max((y-y0)**2)

In [ ]:
co = cheb.getCoeffs()

In [ ]:
xc = -1
y0 = 0


In [ ]:
N = 580
a = cheb.getA()
b = cheb.getB()
x = 2.0
y0 = np.log(x)
xc = 2. * (x - a) / (b- a) -1

y = co[0] + co[1] * xc
Tm2 = 1
Tm1 = xc
print(y)
for i in range(2,N):
    Tx = 2 * Tm1 * xc - Tm2
    y += co[i] * Tx
    Tm2 = Tm1
    Tm1 = Tx

print(np.abs((y**2-y0**2)))

In [ ]:
y